In [18]:
import yaml
import torch
from IPython.display import Image, clear_output  # to display images
import argparse
import os

In [ ]:
logger = logging.getLogger(__name__)


In [2]:
import argparse
import logging
import math
import os
import random
import time
from pathlib import Path
from threading import Thread

import numpy as np
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.experimental import attempt_load
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, labels_to_image_weights, init_seeds, \
    fitness, strip_optimizer, get_latest_run, check_dataset, check_file, check_git_status, check_img_size, \
    check_requirements, print_mutation, set_logging, one_cycle, colorstr
from utils.google_utils import attempt_download
from utils.loss import ComputeLoss
from utils.plots import plot_images, plot_labels, plot_results, plot_evolution
from utils.torch_utils import ModelEMA, select_device, intersect_dicts, torch_distributed_zero_first

ModuleNotFoundError: No module named 'models.common'

In [ ]:
clear_output()
print('Setup complete. Using torch %s %s' % 
      (torch.__version__, 
       torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU')
     )

In [ ]:

%cd D:\diploma\models\

yaml_path = r'D:\diploma\models\data.yaml'
with open(yaml_path, 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
!python train.py --img 416 
                --batch 64 
                --epochs 100 
                --data "D:\diploma\Diploma project\Dataset\data.yaml" 
                --cfg ./models/yolov5s_custom.yaml 
                --weights '' 
                --name YoloBaseline 
                --nosave 
                --device 0

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--weights', type=str, default='yolov5s.pt', help='initial weights path')
parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
parser.add_argument('--data', type=str, default='data/coco128.yaml', help='data.yaml path')
parser.add_argument('--hyp', type=str, default='data/hyp.scratch.yaml', help='hyperparameters path')
parser.add_argument('--epochs', type=int, default=300)
parser.add_argument('--batch-size', type=int, default=16, help='total batch size for all GPUs')
parser.add_argument('--img-size', nargs='+', type=int, default=[640, 640], help='[train, test] image sizes')
parser.add_argument('--rect', action='store_true', help='rectangular training')
parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
parser.add_argument('--notest', action='store_true', help='only test final epoch')
parser.add_argument('--noautoanchor', action='store_true', help='disable autoanchor check')
parser.add_argument('--evolve', action='store_true', help='evolve hyperparameters')
parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
parser.add_argument('--cache-images', action='store_true', help='cache images for faster training')
parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
parser.add_argument('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')
parser.add_argument('--log-imgs', type=int, default=16, help='number of images for W&B logging, max 100')
parser.add_argument('--log-artifacts', action='store_true', help='log artifacts, i.e. final trained model')
parser.add_argument('--workers', type=int, default=8, help='maximum number of dataloader workers')
parser.add_argument('--project', default='runs/train', help='save to project/name')
parser.add_argument('--name', default='exp', help='save to project/name')
parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
parser.add_argument('--quad', action='store_true', help='quad dataloader')


_StoreTrueAction(option_strings=['--quad'], dest='quad', nargs=0, const=True, default=False, type=None, choices=None, help='quad dataloader', metavar=None)

In [ ]:
opt = parser.parse_args()

In [16]:
class ModelParameters:
    
    def __init__(self,
                weights = 'yolov5s.pt',
                cfg = '',
                data='data/coco128.yaml',
                hyp='data/hyp.scratch.yaml',
                epochs=300,
                batch=16,
                img_size=[640, 640],
                rect = True,
                resume=False,
                nosave=True,
                notest=True,
                noautoanchor=True,
                evolve=True,
                bucket='',
                cache_images=True,
                image_weights=True,
                device='',
                multi_scale=True,
                single_cls=True,
                adam=True,
                sync_bn=True,
                local_rank=-1,
                log_imgs=16,
                log_artifacts=True,
                workers=8,
                project='runs/train',
                name='exp',
                exist_ok=True,
                quad=True):
        
        self.weights = weights
        self.cfg = cfg
        self.data = data
        self.hyp = hyp
        self.epochs = epochs
        self.batch = batch
        self.img_size = img_size
        self.rect = rect
        self.resume = resume
        self.nosave = nosave
        self.notest = notest
        self.noautoanchor = noautoanchor
        self.evolve = evolve
        self.bucket = bucket
        self.cache_images = cache_images
        self.image_weights = image_weights
        self.device = device
        self.multi_scale = multi_scale
        self.single_cls = single_cls
        self.adam = adam
        self.sync_bn = sync_bn
        self.local_rank = local_rank
        self.log_imgs = log_imgs
        self.log_artifacts = log_artifacts
        self.workers = workers
        self.project = project
        self.name = name
        self.exist_ok = exist_ok
        self.quad = quad

In [19]:
opt = ModelParameters()

# Set DDP variables
opt.world_size = int(os.environ['WORLD_SIZE']) if 'WORLD_SIZE' in os.environ else 1
opt.global_rank = int(os.environ['RANK']) if 'RANK' in os.environ else -1
set_logging(opt.global_rank)
if opt.global_rank in [-1, 0]:
    check_git_status()
    check_requirements()

# Resume
if opt.resume:  # resume an interrupted run
    ckpt = opt.resume if isinstance(opt.resume, str) else get_latest_run()  # specified or most recent path
    assert os.path.isfile(ckpt), 'ERROR: --resume checkpoint does not exist'
    apriori = opt.global_rank, opt.local_rank
    with open(Path(ckpt).parent.parent / 'opt.yaml') as f:
        opt = argparse.Namespace(**yaml.load(f, Loader=yaml.FullLoader))  # replace
    opt.cfg, opt.weights, opt.resume, opt.batch_size, opt.global_rank, opt.local_rank = '', ckpt, True, opt.total_batch_size, *apriori  # reinstate
    logger.info('Resuming training from %s' % ckpt)
else:
    # opt.hyp = opt.hyp or ('hyp.finetune.yaml' if opt.weights else 'hyp.scratch.yaml')
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
    assert len(opt.cfg) or len(opt.weights), 'either --cfg or --weights must be specified'
    opt.img_size.extend([opt.img_size[-1]] * (2 - len(opt.img_size)))  # extend to 2 sizes (train, test)
    opt.name = 'evolve' if opt.evolve else opt.name
    opt.save_dir = increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok | opt.evolve)  # increment run

# DDP mode
opt.total_batch_size = opt.batch_size
device = select_device(opt.device, batch_size=opt.batch_size)
if opt.local_rank != -1:
    assert torch.cuda.device_count() > opt.local_rank
    torch.cuda.set_device(opt.local_rank)
    device = torch.device('cuda', opt.local_rank)
    dist.init_process_group(backend='nccl', init_method='env://')  # distributed backend
    assert opt.batch_size % opt.world_size == 0, '--batch-size must be multiple of CUDA device count'
    opt.batch_size = opt.total_batch_size // opt.world_size

# Hyperparameters
with open(opt.hyp) as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)  # load hyps

# Train
logger.info(opt)
try:
    import wandb
except ImportError:
    wandb = None
    prefix = colorstr('wandb: ')
    logger.info(f"{prefix}Install Weights & Biases for YOLOv5 logging with 'pip install wandb' (recommended)")
if not opt.evolve:
    tb_writer = None  # init loggers
    if opt.global_rank in [-1, 0]:
        logger.info(f'Start Tensorboard with "tensorboard --logdir {opt.project}", view at http://localhost:6006/')
        tb_writer = SummaryWriter(opt.save_dir)  # Tensorboard
    train(hyp, opt, device, tb_writer, wandb)

NameError: name 'set_logging' is not defined